In [ ]:
%matplotlib inline

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import numpy.random as random

In [ ]:
def roll_dice(dice_config, verbose=False):
    '''
    This is a clever way to take D&D terms and roll that many dice.
    '''
    
    n,sides = dice_config.split("d")
    n = int(n)
    sides = int(sides)
    rolled_dice = np.random.randint(1,high=sides+1,size=n)
    
    if verbose:
        print(rolled_dice)
    
    return rolled_dice

dice_config = "3d4"
print(roll_dice("3d12"))
print(roll_dice("6d6").sum())

In [ ]:
class Item():
    def __init__(self, name, value=None):
        self.name = name
        self.value = None

class Weapon(Item):
    def __init__(self, name, damage_dice, value=None, **kwargs):
        super().__init__(name, value) #Python3    
        self.damage_dice = damage_dice
        self.hit_bonus = 0
        self.damage_bonus = 0
        
        if 'hit_bonus' in kwargs:
            self.hit_bonus = kwargs['hit_bonus']
        if 'damage_bonus' in kwargs:
            self.damage_bonus = kwargs['damage_bonus']

class Armor(Item):
    def __init__(self, name, armor_class, value=None):
        super().__init__(name, value)
        self.armor_class = armor_class
        
class Potion(Item):
    def __init__(self, name, potency, value=None):
        super().__init__(name, value)
        self.potency = potency

In [ ]:
class Creature():
    def __init__(self, name, hp, inventory=None):
        self.name = name
        self.hp = hp
        self.thac0 = 20
        
        self.hit_bonus = 0
        self.damage_bonus = 0
        
        self.weapon = None
        self.armor = None
        
        self.inventory = [] #Empty inventory
        
    def get_armor_class(self):
        if self.armor is not None:
            return self.armor.armor_class
        else:
            return 10
        
    def attack(self, target):
        
        hit_bonus = self.hit_bonus
        if self.weapon is not None:
            hit_bonus += self.weapon.hit_bonus       
        
        #We need to roll this number to hit the target. 
        to_hit = self.thac0 - target.get_armor_class() - hit_bonus      
        
        attack_roll = roll_dice("1d20").sum()
        
        print("\tTo hit: {0:d}\n\tAttack roll: {1:d}".format(to_hit, attack_roll))
        
        if attack_roll >= to_hit:
            if self.weapon is None:
                #Claw the target!
                damage = int(roll_dice("1d3").sum())
            else:
                #Use your weapon!
                damage = roll_dice(self.weapon.damage_dice).sum() + \
                         self.weapon.damage_bonus + \
                         self.damage_bonus
                
            weapon_name = "claws" if self.weapon is None else self.weapon.name   
            print("{0:s} hit target {1:s} with {3:s} for {2:d} damage!".format(self.name, target.name, damage, weapon_name))                
            target.take_damage(damage)
        else:
            print("Missed target!")
                
                
    def take_damage(self, damage):
        self.hp -= damage
        
    def heal(self, hp):
        self.hp += hp        
        
    def show_inventory(self):
        for i in self.inventory:
            print(i)

class Hero(Creature):
    def __init__(self, name, hp, race, char_class):
        #super(Hero, self).__init__(name,hp) #Python 2.7
        super().__init__(name, hp) #Python3
        
        self.race = race
        self.char_class = char_class
               
        self.bonus_tracker = 0
        
    def attack(self, target):
        print("ATTACK!")
        super().attack(target)

        #Every three attacks a hero gets a bonus attack. 
        self.bonus_tracker +=1 
        print(self.bonus_tracker)
        if self.bonus_tracker % 3 == 0 and self.bonus_tracker != 0:
            self.bonus_tracker = 0
            print("Bonus attach for {0:s}".format(self.name))
            super().attack(target)


        
class Monster(Creature):
    def __init__(self, name, hp, species):
        #super(Monster, self).__init__(name,hp) #Python 2.7
        super().__init__(name, hp) #Python3

        self.species = species

        
    def get_armor_class(self):
        '''
        Our monster's AC will be a little bit different. 
        We'll say they're a littl bit better armored (thick skin?)
        So their base armor is 8 in stead of 10, making them harder to hit.
        '''
        if self.armor is not None:
            return self.armor.armor_class - 2
        else:
            return 8
  

In [ ]:
      
a = Monster("James", 100, "Goblin")
b = Hero("Peters", 100, "Elf", "Fighter")

b.weapon = Weapon("Battleaxe", "2d6", value=300, kwargs={'hit_bonus': 1, 'damage_bonus' : 1})
a.weapon = Weapon("Sword", "1d8", value=500, kwargs={'hit_bonus': 2, 'damage_bonus': 2})

In [ ]:
b.attack(a)
a.hp

In [ ]:


def battle(hero):
    
    baddie = Monster("James", roll_dice("3d8").sum(), "Goblin")
    baddie.weapon = Weapon("Bad Guy Sword", "2d4", value=100, kwargs={'hit_bonus': 2, 'damage_bonus': 2})
    
    print("A monster is here!")
    while(True):
        #what other options could we implement here? Take a potion? 
        #We would need to check and see if the hero has a potion in their inventory.
        print("Choose an action!")
        print("\t 1) Attack foe.")
        print("\t 2) Run away!")
        action = input()
        if int(action) == 1:
            hero.attack(baddie)
            if baddie.hp < 0:
                print("You won!")
                return 1
        
        elif int(action) == 2:
            #Here would be a good place to put in a random chance of 
            #succeeding to run away!
            print("Ran away!")
            return 0
        print("")
        print("Enemy attacks!!")
        baddie.attack(hero)
        if hero.hp < 0:
            print("You have perished!")
            return -1
        
def play_game(n_foes_to_beat = 2):
    n_defeated_foes = 0
    
    
    
    print("You must defeat {0:d} monsters in order to win.".format(n_foes_to_beat))
    print("What is your name, hero?")
    name = str(input())
    print("What is your species?")
    race = str(input())
    print("What is your class?")
    char_class = str(input())
    
    hero = Hero(name, roll_dice("3d10").sum(), race, char_class)
    hero.weapon = Weapon("Glamdring", "3d6", value=1000, kwargs={'hit_bonus': 2, 'damage_bonus': 2})
    while True:

        print("Choose an action:")
        print("\t 1) Go looking for trouble.")
        print("\t 2) Quit the game.")
        choice = input()
        if int(choice) == 1:
            fight_result = battle(hero)
            if fight_result < 0:
                print("You died! Game over, man!")
                return 
            else:
                n_defeated_foes += fight_result
        else:
            print("Quitting...")
            return


In [ ]:
play_game()

The next step would be to move the code into a python module and run it from the command line.
To do this, we need to define a main() function and add code so that when the script is run, things are executed:

```
    def main():
        play_game()

    if __name__ == "__main__":
        # execute only if run as a script
        main()

```


